In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [2]:
key_path = key_path
project = project_id
dataset = 'bronze'
table = 'bronze_currency_data'
table_conca = f'{project}.{dataset}.{table}'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
import yfinance as yf
import pandas as pd
from datetime import date


# Definir los pares de divisas para las 10 monedas más importantes
tickers = {
    'Euro (EURUSD)': 'EURUSD=X',
    'Yen Japonés (JPYUSD)': 'JPYUSD=X',
    'Libra Esterlina (GBPUSD)': 'GBPUSD=X',
    'Franco Suizo (USDCHF)': 'USDCHF=X',
    'Dólar Canadiense (USDCAD)': 'USDCAD=X',
    'Dólar Australiano (AUDUSD)': 'AUDUSD=X',
    'Dólar Neozelandés (NZDUSD)': 'NZDUSD=X',
    'Dólar de Hong Kong (USDHKD)': 'USDHKD=X',
    'Dólar de Singapur (USDSGD)': 'USDSGD=X',
    'Rupia India (USDINR)': 'USDINR=X'
}

# Lista para almacenar los DataFrames individuales
df_list = []

# Obtener datos históricos para cada ticker
for name, ticker in tickers.items():
    data = yf.download(ticker, start='2015-01-01', end=date.today())
    data['Currency'] = name  # Añadir columna con el nombre de la moneda
    data.reset_index(inplace=True)  # Convertir el índice 'date' a una columna
    df_list.append(data)

# Concatenar todos los DataFrames en uno solo
df = pd.concat(df_list)

# Imprimir los primeros datos del DataFrame combinado
print("Datos históricos combinados:")
df.head()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Datos históricos combinados:


,Date,Open,High,Low,Close,Adj Close,Volume,Currency
0,2015-01-01,1.209863,1.209863,1.209863,1.209863,1.209863,0,Euro (EURUSD)
1,2015-01-02,1.208868,1.208956,1.201080,1.208941,1.208941,0,Euro (EURUSD)
2,2015-01-05,1.195500,1.197590,1.188909,1.194643,1.194643,0,Euro (EURUSD)
3,2015-01-06,1.193830,1.197000,1.188693,1.193902,1.193902,0,Euro (EURUSD)
4,2015-01-07,1.187479,1.190000,1.180401,1.187536,1.187536,0,Euro (EURUSD)


In [5]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df, project, dataset, table, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
